<center><br><font size=6>Final Project</font><br>
<font size=5>Advanced Topics in Deep Learning</font><br>
<b><font size=4>Part B</font></b>
<br><font size=4>Load Final Models</font><br><br>
Authors: Ido Rappaport & Eran Tascesme
</font></center>

**Submission Details:**
<font size=2>
<br>Ido Rappaport, ID: 322891623
<br>Eran Tascesme , ID: 205708720 </font>


**Import libraries**

❗Note the versions of the packages, we have included information in requirements.txt❗

In [3]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [4]:
# Standard libraries
import os
import random
import warnings
import time

# Data handling and visualization
import pandas as pd
import numpy as np

# Machine learning and deep learning
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
    classification_report,
    confusion_matrix,
    matthews_corrcoef,
    normalized_mutual_info_score,
    ConfusionMatrixDisplay
)

# Hugging Face Transformers
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from transformers.modeling_outputs import SequenceClassifierOutput
from peft import PeftModel

# Filter warnings
warnings.filterwarnings('ignore')


C:\Users\Eran\anaconda3\envs\DeepL\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\Eran\anaconda3\envs\DeepL\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Load CSV Files**


In [6]:
test_data = pd.read_csv("data/test_clean.csv", encoding="ISO-8859-1")
path_dir = "final_models/"

**Load Models**

Load the models and create a list containing all of them. Each model type has its own specific loading function.

In [8]:
model_list = []

In [9]:
# function that load the Base Models who trained by exc4 and exc5 without compression (kivutz)
def load_base_model(model_name, base_weights_path):

  model = AutoModelForSequenceClassification.from_pretrained(
      model_name, num_labels=5, ignore_mismatched_sizes=True
  )
  state_dict = torch.load(base_weights_path, map_location="cpu")
  model.load_state_dict(state_dict)

  tokenizer = AutoTokenizer.from_pretrained(model_name)

  return model, tokenizer


# function that load the pruned and distilled models
def load_compressed_model(save_model_path, device="cpu"):
    model_path = os.path.join(save_model_path, "model.pt")
    model = torch.load(model_path, map_location=device, weights_only=False)
    model.eval()

    tokenizer = AutoTokenizer.from_pretrained(save_model_path)
    return model, tokenizer


# function that load the quantized models
def load_quantized_model(model_name, quantized_model_path):
  q_state_path = os.path.join(quantized_model_path, "model.pt")

  tokenizer = AutoTokenizer.from_pretrained(model_name)

  # Rebuild the same base architecture
  loaded_model = AutoModelForSequenceClassification.from_pretrained(
      model_name, num_labels=5, ignore_mismatched_sizes=True
  )

  # Apply the same dynamic quantization to convert Linear -> quantized Linear
  loaded_model = torch.quantization.quantize_dynamic(
      loaded_model, {nn.Linear}, dtype=torch.qint8
  )

  # Now load the quantized weights (keys will match)
  loaded_model.load_state_dict(torch.load(q_state_path, map_location="cpu"))
  loaded_model.eval()

  return loaded_model, tokenizer

**Base Models**

The models who trained by exc4 and exc5 without compression (kivutz)

In [11]:
#roberta
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"

base_weights_path = "final_models/roberta_sentiment_exc4_weights.pt"
model, tokenizer = load_base_model(model_name, base_weights_path)
model_list.append(("roberta_sentiment_exc4", model, tokenizer))

base_weights_path = "final_models/roberta_sentiment_weights.pt"
model, tokenizer = load_base_model(model_name, base_weights_path)
model_list.append(("roberta_sentiment_exc5", model, tokenizer))

# distilbert
model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"

base_weights_path = "final_models/distilbert_exc4_weights.pt"
model, tokenizer = load_base_model(model_name, base_weights_path)
model_list.append(("distilbert_exc4", model, tokenizer))

base_weights_path = "final_models/distilbert_weights.pt"
model, tokenizer = load_base_model(model_name, base_weights_path)
model_list.append(("distilbert_exc5", model, tokenizer))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

**Pruned & Distilled Models**

In [13]:
models_pathes = ["roberta_exc4_pruned", "roberta_pruned", "distilbert_exc4_pruned", "distilbert_pruned",
                 "distilroberta_exc4-base", "distilroberta-base", "tinybert_exc4", "tinybert"]


for model_path in models_pathes:
    save_model_path = os.path.join(path_dir, model_path)
    model, tokenizer = load_compressed_model(save_model_path, device="cpu")

    model_list.append((model_path, model, tokenizer))

**Quantized Models**

In [15]:
#roberta
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
models_pathes = ["roberta_sentiment_exc4_quantized", "roberta_sentiment_quantized"]

for model_path in models_pathes:
  quantized_model_path = os.path.join(path_dir, model_path)
  model, tokenizer = load_quantized_model(model_name, quantized_model_path)

  model_list.append((model_path, model, tokenizer))

#distilbert
model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
models_pathes = ["distilbert_exc4_quantized", "distilbert_quantized"]

for model_path in models_pathes:
  quantized_model_path = os.path.join(path_dir, model_path)
  model, tokenizer = load_quantized_model(model_name, quantized_model_path)

  model_list.append((model_path, model, tokenizer))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

**LoRA Models**

In [17]:
#roberta
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
lora_list = ["roberta_sentiment_exc4_LoRA", "roberta_sentiment_exc5_LoRA"]

for model_path in lora_list:
    save_model_path = os.path.join(path_dir, model_path)

    base_model_reload = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, ignore_mismatched_sizes=True)
    lora_model_reload = PeftModel.from_pretrained(base_model_reload, save_model_path)
    tokenizer = AutoTokenizer.from_pretrained(save_model_path)

    model_list.append((model_path, lora_model_reload, tokenizer))


#distilbert
lora_list = ["distilbert_exc4_LoRA", "distilbert_exc5_LoRA"]
model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"

for model_path in lora_list:
    save_model_path = os.path.join(path_dir, model_path)

    base_model_reload = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, ignore_mismatched_sizes=True)
    lora_model_reload = PeftModel.from_pretrained(base_model_reload, save_model_path)
    tokenizer = AutoTokenizer.from_pretrained(save_model_path)

    model_list.append((model_path, lora_model_reload, tokenizer))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

**Evaluation**

Evaluate all models and calculate a final score for each.

In [19]:
def evaluate_model_metrics(model, tokenizer, test_data, batch_size=32, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """Compute metrics for a single model on test data."""
    # check if the model is quantized and move it to the cpu/gpu .
    is_quantized = any(
        isinstance(m, nn.quantized.dynamic.Linear) or isinstance(m, nn.quantized.Linear)
        for m in model.modules()
    )
    if is_quantized:
      device = "cpu"

    model.to(device)
    model.eval()

    # store the texts and labels
    texts = test_data['text'].tolist()
    labels = torch.tensor(test_data['label'].tolist()).to(device)

    # start calculate the metrics. time, acc, f1, num_parmas and more.
    start_time = time.time()
    all_preds = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = [str(t) for t in texts[i:i+batch_size]]
            encodings = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt').to(device)
            outputs = model(**encodings)

            logits = outputs.logits if hasattr(outputs, 'logits') else outputs[0]
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())    # store the preds
    runtime = time.time() - start_time

    accuracy = accuracy_score(labels.cpu().numpy(), all_preds)
    f1 = f1_score(labels.cpu().numpy(), all_preds, average='macro')

    mcc = matthews_corrcoef(labels.cpu().numpy(), all_preds)
    nit = normalized_mutual_info_score(labels.cpu().numpy(), all_preds)

    conf_matrix = confusion_matrix(labels.cpu().numpy(), all_preds)

    total_params = sum(p.numel() for p in model.parameters())
    nonzero_params = sum(torch.count_nonzero(p).item() for p in model.parameters())

    return {
        'accuracy': accuracy,
        'f1': f1,
        'mcc': mcc,
        'nit': nit,
        'conf_matrix': conf_matrix,
        'runtime_sec': runtime,
        'total_params': total_params,
        'nonzero_params': nonzero_params
    }

def evaluate_and_score_models(model_list, test_data, weights=None, batch_size=32):
    """
    Evaluate multiple HuggingFace models and compute a relative weighted score.

    Args:
        models: list of (name, model, tokenizer)
        test_data: pd.DataFrame with 'text' and 'label'
        weights: dict with weights for metrics
        batch_size: evaluation batch size

    Returns:
        pd.DataFrame with metrics and final weighted score
    """
    if weights is None:
        weights = {'accuracy': 0.4, 'mcc': 0.2, 'nit': 0.2,'runtime': 0.1, 'params': 0.05, 'nonzero_params': 0.05}

    all_metrics = {}

    # Step 1: compute metrics for all models
    for name, model, tokenizer in model_list:
      print(f"Evaluating {name}...")
      try:
          metrics = evaluate_model_metrics(model, tokenizer, test_data, batch_size=batch_size)
          all_metrics[name] = metrics
      except Exception as e:
          print(f"Error evaluating {name}: {e}")

    df = pd.DataFrame(all_metrics).T

    # Step 2: min-max scale each metric (higher is better for final score)
    df_scaled = df.copy()

    # For metrics where lower is better (runtime, total_params, nonzero_params)
    for col in ['runtime_sec', 'total_params', 'nonzero_params']:
        col_normalization = col + "_norm"
        df_scaled[col_normalization] = 1 / ((df[col] - df[col].min()) / (df[col].max() - df[col].min() + 1e-8) + 0.5)

    # Step 3: compute final weighted score
    df_scaled['final_score'] = (
        weights['accuracy'] * df_scaled['accuracy'] +
        weights['mcc'] * df_scaled['mcc'] +
        weights['nit'] * df_scaled['nit'] +
        weights['runtime'] * df_scaled['runtime_sec_norm'] +
        weights['params'] * df_scaled['total_params_norm'] +
        weights['nonzero_params'] * df_scaled['nonzero_params_norm']
    )

    # Sort by final score
    df_scaled = df_scaled.sort_values(by='final_score', ascending=False)

    return df_scaled


In [20]:
# start evaluation
evaluation_df = evaluate_and_score_models(model_list, test_data)
print(evaluation_df.columns)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Evaluating roberta_sentiment_exc4...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Evaluating roberta_sentiment_exc5...
Evaluating distilbert_exc4...
Evaluating distilbert_exc5...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


...

Evaluating roberta_sentiment_quantized...
Evaluating distilbert_exc4_quantized...
Evaluating distilbert_quantized...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Evaluating roberta_sentiment_exc4_LoRA...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Evaluating roberta_sentiment_exc5_LoRA...
Evaluating distilbert_exc4_LoRA...
Evaluating distilbert_exc5_LoRA...
Index(['accuracy', 'f1', 'mcc', 'nit', 'conf_matrix', 'runtime_sec',
       'total_params', 'nonzero_params', 'runtime_sec_norm',
       'total_params_norm', 'nonzero_params_norm', 'final_score'],
      dtype='object')


In [21]:
evaluation_df[['accuracy', 'f1', 'mcc', 'nit', 'runtime_sec', 'total_params', 'final_score']]

,accuracy,f1,mcc,nit,runtime_sec,total_params,final_score
distilbert_quantized,0.749342,0.759337,0.681289,0.498732,52.92968,23854080,0.83656
roberta_sentiment_quantized,0.736177,0.74678,0.665713,0.479057,94.811949,39037440,0.757835
distilbert_pruned,0.747499,0.757465,0.678261,0.493489,78.844361,66957317,0.752405
distilbert_exc5,0.750132,0.76129,0.682747,0.500303,71.512965,66957317,0.75172
distilbert_exc4_quantized,0.604529,0.616722,0.497042,0.322598,44.115964,23854080,0.717075
roberta_sentiment_exc5,0.748289,0.75888,0.681719,0.498854,145.650394,124649477,0.6752
tinybert,0.406793,0.386043,0.316721,0.206525,11.71474,14351813,0.667367
distilbert_exc4_pruned,0.61901,0.633854,0.518997,0.350883,73.033131,66957317,0.645127
tinybert_exc4,0.373354,0.331221,0.288318,0.194163,12.61069,14351813,0.643525
distilbert_exc4,0.630332,0.642991,0.533828,0.346945,78.176526,66957317,0.638113


<center><h1>END</h1></center>
